In [1]:
import pandas as pd
import ast
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
from IPython.display import display, Markdown
from tqdm import tqdm

# Settings
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

#### A. Create two dataset with 2019 and 2024 ECB Guide

In [2]:
# Load the dataset
file_path = 'ecb_guide_comparison.csv'
full_text = pd.read_csv(file_path)

# Convert the string representations of embeddings to lists
full_text['body_embedding'] = full_text['body_embedding'].apply(ast.literal_eval).tolist()

# Remove numbers from sourve
full_text['source_clean'] = full_text['source'].str.replace('\s+\d+', '', regex=True)
full_text['source_clean'] = full_text['source_clean'].str.replace(' > nan.*', '', regex=True)
full_text['source_clean'] = full_text['source_clean'].str.replace('\s+', ' ', regex=True)

# Create two dataframes with the new and old ECB Guide
new_guide = full_text[full_text['file_name'] != '2019_ecb_guide'].copy()
old_guide = full_text[full_text['file_name'] == '2019_ecb_guide'].copy()

#### B. Compare individual paragraphs of the new ECB Guide with the paragraphs from the 2019 Guide

In [3]:
# Initialize the columns
new_guide['best_similarity'] = 0.0
new_guide['matching_text'] = ''

for index, row in tqdm(new_guide.iterrows(), total=new_guide.shape[0]):
    # Calculate cosine similarity with all paragraphs in old_guide
    similarities = cosine_similarity([row['body_embedding']], list(old_guide['body_embedding']))
    
    # Find the index of the highest similarity amd add data to the new guide
    max_index = np.argmax(similarities)
    new_guide.loc[index, 'best_similarity'] = float(similarities[0, max_index])
    new_guide.loc[index, 'matching_text'] = old_guide.iloc[max_index]['body_of_the_text']

100%|██████████████████████████████████████████████████████████████████████████████| 1414/1414 [02:23<00:00,  9.86it/s]


In [5]:
# Create list of Level 2 headings with the highest number of limited matches
idx_1 = new_guide['best_similarity'] < 0.95
idx_2 = new_guide['word_count'] > 30
limited_matches = new_guide[idx_1 & idx_2]

(limited_matches.groupby('source')['heading_1']
                .count()
                .sort_values(ascending=False)
                .reset_index()
                .head(10))

,source,heading_1
0,Credit risk > 4 Definition of default > nan,39
1,Counterparty credit risk > 13 Risks not in effective expected positive exposure > nan,33
2,Credit risk > 6 Loss given default > nan,18
3,Credit risk > 5 Probability of default > nan > Principles specific for grades and pools,17
4,General topics > 2 Roll-out and permanent partial use > nan,12
5,Counterparty credit risk > 3 Margin period of risk and cash flows > nan,9
6,Market risk > 6 Methodology for IRC models focusing on default risk > nan,8
7,Counterparty credit risk > 9 Use test > nan,8
8,Credit risk > 3 Use of data > nan,6
9,General topics > 1 Overarching principles for internal models > nan,6


In [8]:
# Compare number of Level 2 paragraphs in the 2019 and 2024 Guide
outcome = pd.pivot_table(full_text, 
                         values='word_count', 
                         index='source_clean', 
                         columns='file_name', 
                         aggfunc='count', 
                         margins=True).sort_values(by='All', ascending=False).reset_index()
outcome['changes'] = outcome['2024_ecb_guide'] - outcome['2019_ecb_guide']

outcome.head(15)

file_name,source_clean,2019_ecb_guide,2024_ecb_guide,All,changes
0,All,1280.0,1412.0,2692,132.0
1,Market risk > Scope of the internal model approach,89.0,95.0,184,6.0
2,Credit risk > Probability of default,66.0,86.0,152,20.0
3,Market risk > Regulatory back-testing of VaR models,76.0,72.0,148,-4.0
4,Market risk > Methodology for IRC models focusing on default risk,61.0,67.0,128,6.0
5,Credit risk > Loss given default,51.0,77.0,128,26.0
6,Market risk > Methodology for VaR and stressed VaR,59.0,66.0,125,7.0
7,General topics > Overarching principles for internal models,55.0,68.0,123,13.0
8,Credit risk > Use of data,44.0,52.0,96,8.0
9,General topics > Internal validation,55.0,39.0,94,-16.0


In [ ]:
# Random sample of paragraphs from the new Guide with limited matches in the Old Guide
cols = ['file_name', 
        'heading_1', 
        'heading_2', 
        'body_of_the_text', 
        'page_number', 
        'source_clean', 
        'word_count', 
        'best_similarity', 
        'matching_text']

limited_matches[cols].sample(3)

#### C. Compare larger subsections

In [7]:
# Create a single string with the specified format
section = 'Credit risk > Probability of default'
idx_1 = new_guide['source_clean'] == section
idx_2 = old_guide['source_clean'] == section

new_guide_text = '\n\n'.join('\n'.join([row['body_of_the_text']]) for index, row in new_guide[idx_1].iterrows())
old_guide_text = '\n\n'.join('\n'.join([row['body_of_the_text']]) for index, row in old_guide[idx_2].iterrows())

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-0125-preview",
  messages=[
    {"role": "system", "content": "You are a helpful regulatory assistant, who is capable to go into details."},
    {"role": "user", "content": "Please outline the most important differences between the two documents."},
    {"role": "assistant", "content": "Please provide the first document."},
    {"role": "user", "content": f"This is the ECB Guide from 2024: {new_guide_text}"},
    {"role": "assistant", "content": "Please provide the second document."},
    {"role": "user", "content": f"This is the ECB Guide from 2019: {old_guide_text}"}
  ]
)

display(Markdown(response.choices[0].message.content))

Analyzing the documents provided for the ECB Guide from 2019 and the ECB Guide from 2024 reveals several key differences and updates reflective of evolving regulatory expectations and greater detail in the guidelines. Below are some of the most important differences:

### Structure & Content Expansion:

- **Document Structure and Detail**: The 2024 document seems to offer more detailed guidelines and expectations, particularly around the internal models for Probability of Default (PD) estimation, compared to the 2019 version. It introduces more refined principles, indicating an evolution in the regulatory stance towards a more detailed and prescriptive approach.

### Specific Updates and Additions in 2024:

- **Model Performance and Risk Drivers**:
  - The 2024 guide places significant emphasis on model performance across economically significant and material sub-ranges of application, suggesting a deeper dive into model validation practices than the 2019 version. It provides a non-exhaustive list of potential drivers for risk differentiation with clearer definitions, reflecting a nuanced understanding of risk factors across various exposure types.
  
- **Model Misspecification and Overfitting**:
  - Both documents discuss the risk of model misspecification and the importance of avoiding overfitting. However, the 2024 document elaborates on the ECB’s expectations regarding the statistical process for model selection, including the use of independent datasets for assessing model performance, indicating a stronger focus on robust model development practices to mitigate overfitting and misspecification risks.

- **Sub-ranges of Application**:
  - The 2024 guide elaborates more on the performance of PD models within sub-ranges of application, mentioning the need for models to perform adequately at sub-consolidated and/or individual levels. This addition suggests a nuanced approach towards model applicability across different segments and hierarchies within institutions.
  
- **Risk Differentiation and Grade Assignment**:
  - The 2024 version appears to offer a more structured approach towards risk differentiation, the setting of metrics, and the establishment of tolerance levels for deviations. It specifically addresses the dynamics of grade assignment over a longer time horizon, reflecting a forward-looking approach to risk assessment.
  
- **Use of External Data for PD Estimation**:
  - While both versions discuss the use of external data, the 2024 document provides a more detailed exploration of the expectations surrounding the use of external ratings or scores, including the adjustment of external rating scales to ensure they solely embed default risk.

- **Risk Quantification & Calibration**:
  - The 2024 guide offers comprehensive insights into the computation of one-year default rates, observed average default rates, and long-run average (LRA) default rates. It places a stronger emphasis on ensuring that external or pooled data series used for PD quantification are representative and that comparisons between internal and external average observed default rates are diligently performed and documented.

### Regulatory Developments:

- **References to Specific Regulations**: The 2024 document reflects updates in regulatory references, including explicit mentions of specific articles within regulatory texts (e.g., CRR, EBA Guidelines) and more detailed guidance on adhering to these regulations. This suggests an effort to align the guidelines more closely with current regulatory standards and expectations.

Overall, the changes from the 2019 to the 2024 version of the ECB Guide on PD models underscore a maturation in the regulatory landscape, emphasizing more granular risk identification, model validation rigor, and the importance of ensuring models are fit for future conditions and not just past data. These updates likely reflect ongoing developments in financial modeling best practices, technological advancements, and a deeper regulatory focus on safeguarding the banking sector against emerging risks.